# Backtest Helpers Example

This notebook demonstrates how to use the `backtest_helpers` module to run backtests and analyze results.

## 1. Import Helper Functions

## 0. Register Sharadar Bundle

**Important:** Jupyter notebooks don't automatically load bundle extensions, so we need to register the sharadar bundle manually.

In [ ]:
# Register the sharadar bundle (required for Jupyter notebooks)
from zipline.data.bundles import register
from zipline.data.bundles.sharadar_bundle import sharadar_bundle

register(
    'sharadar',
    sharadar_bundle(
        tickers=None,
        incremental=True,
        include_funds=True,
    ),
)

# Verify registration
from zipline.data.bundles import bundles
print(f"✓ Sharadar bundle registered!")
print(f"Available bundles: {list(bundles.keys())}")

In [ ]:
# Import the helper functions
from backtest_helpers import backtest, analyze_results, quick_backtest

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Configure display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)

%matplotlib inline

## 2. Run a Backtest

The `backtest()` function runs a backtest and saves results to disk.

**Parameters matching your example:**
- First parameter: algorithm filename
- Second parameter: backtest name/identifier
- Last parameter: output filename

In [ ]:
# Run backtest with your exact signature
perf = backtest(
    "backtest_with_fundamentals.py",  # Algorithm file
    "quality-strategy-v1",             # Name/identifier
    bundle="sharadar",
    data_frequency='daily',
    progress='D',
    start_date="2023-04-01",
    end_date="2024-01-31",
    capital_base=100000,
    filepath_or_buffer="quality-strategy-v1.csv"  # Output filename
)

## 3. Inspect Results DataFrame

The backtest returns a DataFrame with performance data.

In [ ]:
# View first few rows
print("Results DataFrame:")
print(f"Shape: {perf.shape}")
print(f"Columns: {list(perf.columns)}")
print()
perf.head()

In [ ]:
# View summary statistics
perf[['portfolio_value', 'returns', 'algorithm_period_return']].describe()

## 4. Analyze Results with Pyfolio

The `analyze_results()` function loads saved results and generates comprehensive analysis.

In [ ]:
# Analyze the backtest results
results = analyze_results(
    "./backtest_results/quality-strategy-v1.csv",
    benchmark_symbol="SPY",
    show_plots=True,
    return_dict=True
)

In [ ]:
# View detailed metrics
print("Performance Metrics:")
for metric, value in results['metrics'].items():
    print(f"  {metric}: {value}")

## 5. Run Multiple Backtests

Example of running multiple backtests with different parameters.

In [ ]:
# Define different test scenarios
scenarios = [
    {
        'name': 'short-period',
        'start_date': '2023-01-01',
        'end_date': '2023-06-30',
        'capital_base': 100000,
    },
    {
        'name': 'long-period',
        'start_date': '2022-01-01',
        'end_date': '2023-12-31',
        'capital_base': 100000,
    },
    {
        'name': 'high-capital',
        'start_date': '2023-01-01',
        'end_date': '2023-12-31',
        'capital_base': 1000000,
    },
]

# Run all scenarios
results_dict = {}

for scenario in scenarios:
    name = scenario['name']
    print(f"\n{'='*80}")
    print(f"Running scenario: {name}")
    print(f"{'='*80}\n")
    
    perf = backtest(
        "backtest_with_fundamentals.py",
        name,
        bundle="sharadar",
        start_date=scenario['start_date'],
        end_date=scenario['end_date'],
        capital_base=scenario['capital_base'],
        filepath_or_buffer=f"{name}.csv"
    )
    
    results_dict[name] = perf

## 6. Compare Multiple Backtests

In [ ]:
# Compare cumulative returns
fig, ax = plt.subplots(figsize=(14, 6))

for name, perf in results_dict.items():
    # Calculate cumulative returns
    if 'returns' in perf.columns:
        returns = perf['returns']
    else:
        returns = perf['portfolio_value'].pct_change().fillna(0)
    
    cumulative = (returns + 1).cumprod() - 1
    ax.plot(perf.index, cumulative * 100, label=name, linewidth=2)

ax.set_title('Cumulative Returns Comparison', fontsize=14, fontweight='bold')
ax.set_ylabel('Return (%)')
ax.set_xlabel('Date')
ax.legend()
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='black', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

In [ ]:
# Create comparison table
comparison_data = []

for name, perf in results_dict.items():
    if 'returns' in perf.columns:
        returns = perf['returns']
    else:
        returns = perf['portfolio_value'].pct_change().fillna(0)
    
    total_return = (returns + 1).cumprod()[-1] - 1
    annual_vol = returns.std() * np.sqrt(252)
    sharpe = (returns.mean() / returns.std() * np.sqrt(252)) if returns.std() > 0 else 0
    
    cumulative = (returns + 1).cumprod()
    running_max = cumulative.cummax()
    drawdown = (cumulative - running_max) / running_max
    max_dd = drawdown.min()
    
    comparison_data.append({
        'Scenario': name,
        'Total Return': f"{total_return:.2%}",
        'Annual Vol': f"{annual_vol:.2%}",
        'Sharpe': f"{sharpe:.3f}",
        'Max DD': f"{max_dd:.2%}",
        'Final Value': f"${perf['portfolio_value'].iloc[-1]:,.0f}",
    })

comparison_df = pd.DataFrame(comparison_data)
print("\nBacktest Comparison:")
print(comparison_df.to_string(index=False))

## 7. Quick Backtest

For rapid testing, use `quick_backtest()` to run and analyze in one step.

In [ ]:
# Quick backtest - runs and analyzes automatically
perf = quick_backtest(
    "backtest_with_fundamentals.py",
    name="quick-test",
    start_date="2023-01-01",
    end_date="2023-12-31",
    capital_base=100000,
    bundle="sharadar",
)

## 8. Reload Previous Results

You can analyze previously saved results without re-running the backtest.

In [ ]:
# Analyze a previously saved backtest
analyze_results(
    "./backtest_results/quality-strategy-v1.csv",
    benchmark_symbol="SPY",
    show_plots=True
)

## 9. Custom Analysis

Load results and perform custom analysis.

In [ ]:
# Load results with full details
results = analyze_results(
    "./backtest_results/quality-strategy-v1.csv",
    show_plots=False,
    return_dict=True
)

# Access components
returns = results['returns']
perf = results['perf']
metrics = results['metrics']

# Custom analysis: Rolling Sharpe ratio
rolling_sharpe = (
    returns.rolling(window=60)
    .apply(lambda x: x.mean() / x.std() * np.sqrt(252) if x.std() > 0 else 0)
)

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(rolling_sharpe.index, rolling_sharpe, linewidth=2)
ax.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax.axhline(y=1, color='green', linestyle='--', alpha=0.3, label='Sharpe = 1')
ax.axhline(y=2, color='blue', linestyle='--', alpha=0.3, label='Sharpe = 2')
ax.set_title('60-Day Rolling Sharpe Ratio', fontsize=14, fontweight='bold')
ax.set_ylabel('Sharpe Ratio')
ax.set_xlabel('Date')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Summary

The `backtest_helpers` module provides:

1. **`backtest()`** - Run backtest and save results
   - First param: algorithm filename
   - Second param: backtest name
   - Last param: output filename
   - Saves CSV, pickle, and metadata

2. **`analyze_results()`** - Load and analyze saved results
   - Generates pyfolio tearsheet
   - Creates performance plots
   - Calculates comprehensive metrics

3. **`quick_backtest()`** - Run and analyze in one step
   - Convenient for rapid iteration

All results are saved to `./backtest_results/` by default and can be reloaded anytime.